In [1]:
import os

from langchain_openai import ChatOpenAI
# from langchain_community.llms import Tongyi
from langchain_zhipu import ChatZhipuAI
from illufly.project import WritingProject
from illufly.exporter.pptx import export_pptx

openai = WritingProject(
    project_id="PPT",
    llm=ChatZhipuAI(model_name="gpt-4")
)

zhipu = WritingProject(
    project_id="PPT",
    llm=ChatZhipuAI()
)

In [ ]:
zhipu.idea("帮我生成一份PPT提纲", completed="实习要求.md", output_file="实习PPT.md")

In [7]:
from illufly.importer import load_markdown
from pptx import Presentation
from pptx.util import Inches, Pt

md_old = load_markdown("PPT/实习PPT.md")
old_docs = md_old.documents
m, segments = md_old.get_todo_documents("segment")

In [8]:
old_docs[0]

Document(page_content='', metadata={'modified_at': '2024-07-08 19:39:13', 'output_file': '实习PPT.md', 'command': 'stream', 'args': {'task': '帮我生成一份PPT提纲', 'completed': '实习要求.md', 'sep_mode': 'all', 'prompt_id': 'IDEA', 'tag_start': '```', 'tag_end': '```', 'base_folder': './PPT'}, 'id': '0664-440-000', 'type': 'front_matter'})

In [9]:
segments[0]

[(Document(page_content='', metadata={'modified_at': '2024-07-08 19:39:13', 'output_file': '实习PPT.md', 'command': 'stream', 'args': {'task': '帮我生成一份PPT提纲', 'completed': '实习要求.md', 'sep_mode': 'all', 'prompt_id': 'IDEA', 'tag_start': '```', 'tag_end': '```', 'base_folder': './PPT'}, 'id': '0664-440-000', 'type': 'front_matter'}),
  0),
 (Document(page_content='\n', metadata={'type': 'blank_line', 'id': '0664-448-001'}),
  1)]

In [ ]:
prs = Presentation()

for doc, index in segments:
    slide_layout = prs.slide_layouts[1]  # 0是标题幻灯片，1是标题和内容
    slide = prs.slides.add_slide(slide_layout)

    if last_index != None:
        md = "\n"
        output_text += md
        yield TextChunk('text', md)
    if old_docs.documents[last_index:index]:
        md = MarkdownLoader.to_markdown(old_docs.documents[last_index:index])
        output_text += md
        yield TextChunk('text', md)

    last_index = index + 1
    
    if doc.page_content and doc.page_content.strip():
        _kwargs = {
            "knowledge__": kg_doc,
            "todo_doc__": doc.page_content,
            "prev_doc__": MarkdownLoader.to_markdown(new_docs.get_prev_documents(doc, prev_k)),
            "next_doc__": MarkdownLoader.to_markdown(new_docs.get_next_documents(doc, next_k)),
            "history": "",
            **kwargs
        }
        chain = _create_chain(llm, prompt, **_kwargs)

        resp_md = ""
        for chunk in _call_markdown_chain(chain, {"task": task}, is_fake, verbose):
            if chunk.mode == 'chunk':
                resp_md += chunk.content
            yield chunk

        final_md = extract_text(resp_md, tag_start, tag_end)
        to_insert = new_docs.replace_documents(doc, doc, final_md)
        final_md_strip = MarkdownLoader.to_markdown(to_insert).strip()

        output_text += final_md_strip
        yield TextChunk('final', final_md_strip)

    else:
        # 如果内容是空行就不再处理
        output_text += doc.page_content
        yield TextChunk('text', doc.page_content)

if old_docs.documents[last_index:None]:
    md = MarkdownLoader.to_markdown(old_docs.documents[last_index:None])
    output_text += md
    yield TextChunk('text', md)

for docs, index in segments:
    slide_layout = prs.slide_layouts[1]  # 0是标题幻灯片，1是标题和内容
    slide = prs.slides.add_slide(slide_layout)
    title_set = False

    for doc in docs:
        print('type >>', doc.metadata['type'])
        if doc.metadata['type'] in ['front-matter', 'blank_line']:
            continue

        if old_docs.documents[last_index:index]:
            md = MarkdownLoader.to_markdown(old_docs.documents[last_index:index])

        if doc.metadata['type'] == 'heading':
            title = slide.shapes.title
            title.text = doc.page_content
            title_set = True
            
            print("heading >>", doc.page_content)
        elif doc.page_content:
            if not title_set:  # 如果还没有设置标题，使用第一个内容作为标题
                title = slide.shapes.title
                title.text = doc.page_content
                title_set = True

                print("not title set >>", doc.page_content)
            else:
                content_placeholder = slide.placeholders[1]
                content_text_frame = content_placeholder.text_frame
                if content_text_frame.text:  # 如果已经有文本，添加新行
                    content_text_frame.add_paragraph().text = doc.page_content
                else:  # 如果没有文本，直接设置文本
                    content_text_frame.text = doc.page_content
            
                print("text >>", doc.page_content)
                print(doc.page_content)


In [ ]:
len(prs.slides)

In [ ]:
%pwd

In [ ]:
export_pptx("PPT/实习PPT.md")